In [27]:
import pandas as pd
import numpy as np
import os, shutil
from shutil import copyfile
import matplotlib.pyplot as plt
from pathlib import Path
from hashlib import md5
from collections import defaultdict
import PIL
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras_preprocessing.image import load_img

from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras import regularizers
from tensorflow.keras.regularizers import l2
# from tensorflow.keras.applications import EfficientNetB0
from keras.optimizers import RMSprop
from pathlib import Path
import random
import os
import os.path
from PIL import Image
from sklearn.model_selection import train_test_split
import matplotlib
import matplotlib.pyplot as plt
from scipy import signal
import plotly.express as px
import collections
SEED=42

In [ ]:
# !kaggle competitions download -c kitchenware-classification
# !mkdir data
# !unzip kitchenware-classification.zip -d data > /dev/null
# !rm kitchenware-classification.zip

In [48]:
project_dir = Path('./Images/')
img_dir = project_dir / 'images'

In [3]:
df = pd.read_csv('./data/train.csv')
df["Id"]
df['Id'] = df['Id'].astype(str).str.zfill(4)
df

,Id,label
0,0560,glass
1,4675,cup
2,0875,glass
3,4436,spoon
4,8265,plate
...,...,...
5554,6742,cup
5555,1574,cup
5556,6746,knife
5557,3562,cup


In [4]:
df['filename'] = df.apply(lambda row: str(row['Id']) + ".jpg", axis=1)

In [5]:
df = df.drop(['Id'], axis=1)

In [6]:
first_column = df.pop('filename')
df.insert(0, 'filename', first_column)
df = df.reset_index(drop=True)
df

,filename,label
0,0560.jpg,glass
1,4675.jpg,cup
2,0875.jpg,glass
3,4436.jpg,spoon
4,8265.jpg,plate
...,...,...
5554,6742.jpg,cup
5555,1574.jpg,cup
5556,6746.jpg,knife
5557,3562.jpg,cup


In [7]:
print(len(df))

5559


In [8]:
df_train, df_val = train_test_split(df, test_size=0.2, random_state=SEED)

In [9]:
print(len(df_train)), print(len(df_val))
print(len(df_train)*100/len(df)), print(len(df_val)*100/len(df))

4447
1112
79.99640223061702
20.00359776938298


(None, None)

In [10]:
# Reset indices of all partitions 
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

In [11]:
df_train.head()

,filename,label
0,8242.jpg,spoon
1,0181.jpg,knife
2,4897.jpg,spoon
3,7430.jpg,spoon
4,4286.jpg,glass


In [12]:
df_val.head()

,filename,label
0,4016.jpg,plate
1,4991.jpg,knife
2,1458.jpg,spoon
3,9066.jpg,spoon
4,5767.jpg,cup


In [13]:
df_train.to_csv('df_train.csv')
df_val.to_csv('df_val.csv')

In [45]:
base_dir = r'Images'

In [53]:
import os
import shutil
import pandas as pd

def move_images(img_dir, df_train, df_val, base_dir):
    # Create the train and val directories
    train = os.path.join(base_dir, 'train')
    os.makedirs(train, exist_ok=True)

    val = os.path.join(base_dir, 'val')
    os.makedirs(val, exist_ok=True)

    # Iterate through the rows in the training DataFrame
    for _, row in df_train.iterrows():
        filename = row['filename']
        label = row['label']
        # Create the subdirectory for the label, if it doesn't exist
        subdir = os.path.join(train, label)
        if not os.path.exists(subdir):
            os.mkdir(subdir)

        # Move the image to the subdirectory
        src_path = os.path.abspath(os.path.join(img_dir, filename)).replace("\\","/")
        dest_path = os.path.abspath(os.path.join(subdir, filename)).replace("\\","/")
        shutil.copy(src_path, dest_path)

    # Iterate through the rows in the validation DataFrame
    for _, row in df_val.iterrows():
        filename = row['filename']
        label = row['label']
        # Create the subdirectory for the label, if it doesn't exist
        subdir = os.path.join(val, label)
        if not os.path.exists(subdir):
            os.mkdir(subdir)

        # Move the image to the subdirectory
        src_path = os.path.abspath(os.path.join(img_dir, filename)).replace("\\","/")
        dest_path = os.path.abspath(os.path.join(subdir, filename)).replace("\\","/")
        shutil.copy(src_path, dest_path)


In [54]:
move_images(img_dir, df_train, df_val, base_dir)

In [40]:
df_test = pd.read_csv('data/test.csv', dtype={'Id': str})
df_test['filename'] = df_test['Id'] + '.jpg'
df_test.head()

,Id,filename
0,0678,0678.jpg
1,3962,3962.jpg
2,9271,9271.jpg
3,5133,5133.jpg
4,8842,8842.jpg


In [41]:
import os
import shutil

def test_images(img_dir, df_test, base_dir):
    # Create the test directory
    test = os.path.join(base_dir, 'test')
    os.makedirs(test, exist_ok=True)

    # Iterate through the rows in the DataFrame
    for _, row in df_test.iterrows():
        filename = row['filename']
        # Create the subdirectory for the label, if it doesn't exist
        subdir = os.path.join(test)
        if not os.path.exists(subdir):
            os.mkdir(subdir)

        # Move the image to the subdirectory
        src_path = os.path.abspath(os.path.join(img_dir, filename)).replace("\\","/")
        dest_path = os.path.abspath(os.path.join(subdir, filename)).replace("\\","/")
        shutil.copy(src_path, dest_path)


In [42]:
test_images(img_dir, df_test, base_dir)